# Protegendo os Dados do Cliente com um Algoritmo de Transformação

A empresa de seguros Proteja Seu Amanhã busca proteger os dados de seus clientes desenvolvendo um algoritmo de transformação que dificulte a recuperação de informações pessoais. O desafio é garantir a segurança dos dados sem comprometer a qualidade dos modelos de aprendizado de máquina. O objetivo é criar um algoritmo eficaz e demonstrar sua efetividade por meio de uma prova matemática. Dessa forma, a Proteja Seu Amanhã poderá preservar a privacidade dos clientes, mantendo a precisão e confiabilidade das análises de dados.

# Sumário
1. [Iniciação](#in)
2. [Multiplicação de Matrizes](#mm)
3. [Algoritmo de Transformação](#atr)
4. [Algoritmo de Teste](#ate)
5. [Conclusão](#cc)

## Iniciação <a name="step1"></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
try:
    data = pd.read_csv('insurance_us.csv')
except:
    data = pd.read_csv('/datasets/insurance_us.csv')

In [3]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              5000 non-null   int64  
 1   Age                 5000 non-null   float64
 2   Salary              5000 non-null   float64
 3   Family members      5000 non-null   int64  
 4   Insurance benefits  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Gender,Age,Salary,Family members,Insurance benefits
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Não há valores ausentes. Agora verificaremos se existem linhas duplicadas.

In [4]:
data.duplicated().sum()

153

Existem 153 linhas duplicadas. Não é necessariamente preciso removê-las, dado que o objetivo é verificar se uma transformação de matriz altera ou não a qualidade do aprendizado de máquina.

## Multiplicação de Matrizes <a name="mm"></a>

- $X$ — matriz de características (a coluna zero consiste de unidades)

- $y$ — vetor alvo

- $P$ — matriz pela qual as características são multiplicadas

- $w$ — vetor de pesos da regressão linear (o elemento zero é igual ao deslocamento)

Predições:

$$
a = Xw
$$

Objetivo de treinamento:

$$
\min_w d_2(Xw, y)
$$

Fórmula de treinamento:

$$
w = (X^T X)^{-1} X^T y
$$

__Solução__:

Na fórmula de treinamento, podemos multiplicar a matriz de características pela matriz P para criar um algoritmo que torne mais difícil recuperar informações pessoais a partir dos dados transformados. Tentaremos criar uma prova matemática para demonstrar isso.

$$ X'=XP $$

Denominaremos as novas previsões como a'. A equação de previsão é então modificada da seguinte forma: $$ a'=X'w'=XPw' $$

O novo peso do vetor w' será: $$ w'=(X'^T X')^{-1}X'^Ty=(X^TP^TXP)^{-1}X^TP^Ty $$


__Justificação:__

Substituindo o valor de w' em a': $$ a'=XP(X^TP^TXP)^{-1}X^TP^Ty $$

Rearranjando os termos: $$ a'=X(PP^{-1})(X^TX)^{-1}(P^T)^{-1}P^TX^Ty $$

Qualquer matriz multiplicada por sua inversa resulta na matriz identidade, deste modo, simplificamos: 
$$ a'=X(X^TX)^{-1}X^Ty=Xw=a $$

Isso comprova que as previsões não serão alteradas e a qualidade do modelo de aprendizado de máquina não será prejudicada.

A relação entre w' e w:
$$ w'=(X^TP^TXP)^{-1}X^TP^Ty=(P^TP)^{-1}P^T(X^TX)^{-1}X^Ty=P^{-1}w $$

## Algoritmo de Transformação <a name="atr"></a>

__Algoritmo__
1. Criar uma matriz de transformação aleatória. Ela deve ser uma matriz quadrada, com sua forma sendo o número de colunas de características. Por exemplo: `t_matrix=np.random.normal(size=(features_columns, features_columns))`

2. Verificar se a matriz aleatória é invertível usando: `np.linalg.inv(t_matrix)`. Se recebermos um erro desta operação, significa que a matriz não é invertível, mas esse caso é bastante raro.

3. Multiplicar a matriz de características pela matriz aleatória invertível.

4. Obter os `r2_scores` para os modelos antes e depois da transformação.

__Justificação__

A prova de que a ofuscação dos dados não afetará a qualidade do modelo será demonstrada se as pontuações de R2 para os modelos antes e depois da transformação forem idênticos (ou apresentarem uma diferença insignificante).

## Algoritmo de Teste <a name="ate"></a>

Antes de tudo, dividiremos os dados.

In [5]:
features = data.drop('Insurance benefits', axis=1) 
target = data['Insurance benefits']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=12345)

Podemos criar a matriz de transformação, na qual o número de linhas e o número de colunas são iguais ao número de características.

In [6]:
t_matrix = np.random.normal(size=(x_train.shape[1], x_train.shape[1]))
t_matrix

array([[-0.22364295, -0.51529631, -1.63478913,  0.27938864],
       [ 0.19333591, -1.11601689,  0.02239367,  1.75963434],
       [-1.38215663, -0.88800522,  0.55072179,  1.74660307],
       [-0.90813379, -0.27757844,  0.19676916,  0.47206562]])

Verificaremos se a matriz de transformação aleatória é invertível calculando sua inversa.

In [7]:
np.linalg.inv(t_matrix)

array([[ -0.13382751,   0.36116432,  -0.12660126,  -0.79862859],
       [  0.91201973,  -4.02666994,   7.14049376, -11.9495031 ],
       [ -0.77780655,   0.87181353,  -1.45390289,   2.58995467],
       [  0.60303496,  -2.03632126,   4.56114448,  -7.52397361]])

A matriz é invertível, então podemos prosseguir e transformar os dados realizando uma multiplicação de matriz das características pela matriz de transformação.

In [8]:
x_train_trans = x_train@t_matrix
x_test_trans = x_test@t_matrix

Agora podemos treinar o modelo de regressão linear e comparar o `r2_score` para os modelos de antes e depois da transformação.

In [9]:
model = LinearRegression()

#Antes
model.fit(x_train, y_train)
pred1 = model.predict(x_test)
r2_before = r2_score(y_test, pred1)
rmse_before = (mean_squared_error(y_test, pred1))**0.5

#Depois
model.fit(x_train_trans, y_train)
pred2 = model.predict(x_test_trans)
r2_after = r2_score(y_test, pred2)
rmse_after = (mean_squared_error(y_test, pred2))**0.5

print('-----ANTES DA TRANSFORMAÇÃO-----')
print('R2 score: {:.2f}, RMSE: {:.2f}'.format(r2_before, rmse_before))
print('-----DEPOIS DA TRANSFORMAÇÃO-----')
print('R2 score: {:.2f}, RMSE: {:.2f}'.format(r2_after, rmse_after))

-----ANTES DA TRANSFORMAÇÃO-----
R2 score: 0.41, RMSE: 0.33
-----DEPOIS DA TRANSFORMAÇÃO-----
R2 score: 0.41, RMSE: 0.33


As diferenças entre as pontuação R2 e RMSE são muito insignificantes. Portanto, a ofuscação não afetou o modelo.

## Conclusão <a name="cc"></a>
1. Provamos teoricamente que a transformação não afeta a fórmula de previsão.
2. Treinamos modelos de regressão linear para o cenário sem transformação e com transformação.
3. Obtivemos as pontuações de R2 e RMSE para ambos os cenários e observamos que as diferenças entre eles são insignificantes, provando que a transformação não afeta a qualidade do modelo.